In [75]:
import numpy as np
import json
import re
import sys

In [76]:
CTAGS_FNS = "./ctags.txt"
PERF_FOLDED = "perf_results/C230_FR.cnf-perf.log"

In [77]:
target_functions = []
for line in open(CTAGS_FNS, "r"):
    target_functions.append(line.split(maxsplit=1)[0])
target_functions

['allocate_manager_keys',
 'allocate_vtree_keys',
 'bits2bytes',
 'c2d_version',
 'c2d_version',
 'clist_size',
 'compile_dispatcher',
 'compile_dispatcher',
 'compile_vtree',
 'compile_vtree',
 'compile_vtree_decomposed',
 'compile_vtree_leaf',
 'compile_vtree_shannon',
 'compile_vtree_shannon',
 'compile_with_literal',
 'construct_vtree_cache',
 'construct_vtree_key',
 'construct_vtree_key',
 'copy_key',
 'copy_key',
 'count_dispatcher',
 'count_dispatcher',
 'count_vtree',
 'count_vtree',
 'count_vtree_decomposed',
 'count_vtree_leaf',
 'count_vtree_shannon',
 'count_vtree_shannon',
 'count_with_literal',
 'drop_cache_entry',
 'drop_vtree_cache_entries',
 'drop_vtree_cache_entries',
 'drop_vtree_cache_entries',
 'extended_file_name',
 'extended_file_name',
 'free_cache_entry',
 'free_manager_keys',
 'free_vtree_cache',
 'free_vtree_keys',
 'get_options',
 'get_options',
 'init_options',
 'insert_cache',
 'insert_cache',
 'insert_cache',
 'lookup_cache',
 'lookup_cache',
 'lookup_cac

In [78]:
def match_main_line(line):
    children_pct_pattern = r"\s*(?P<children_pct>\d+\.\d+)%"
    self_pct_pattern = r"\s+(?P<self_pct>\d+\.\d+)%"
    command_pattern = r"\s+(?P<command>\S+)"
    sharedobject_pattern = r"\s+(?P<sharedobject>\S+)"
    symbol_pattern = r"\s+\[\.\]\s+(?P<symbol>\S+)"
    pattern = (
        "^" +
        children_pct_pattern +
        self_pct_pattern +
        command_pattern +
        sharedobject_pattern +
        symbol_pattern +
        "$"
    )

    match = re.match(pattern, line)
    if match:
        res = match.groupdict()
        res['children_pct'] = float(res['children_pct'])
        res['self_pct'] = float(res['self_pct'])
        return res
    else:
        return None

In [79]:
main_lines = []
for line in open(PERF_FOLDED, "r"):
    res = match_main_line(line)
    if res:
        main_lines.append(res)
main_lines = sorted(main_lines, key=lambda x: x["self_pct"], reverse=True)
for ml in main_lines:
    if ml["self_pct"] == 0:
        continue
    print(ml["symbol"], ml["self_pct"])

total_self_time = 0
for ml in main_lines:
    total_self_time += ml["self_pct"]
print("total", total_self_time)

total_minic2d_time = 0
for ml in main_lines:
    source = ml["symbol"]
    if source in target_functions:
        total_minic2d_time += ml["self_pct"]
print("minic2d", total_minic2d_time)

construct_vtree_key 21.61
sat_is_subsumed_clause 15.65
get_and_node 11.72
sat_is_implied_literal 10.77
lookup_cache 4.77
sat_var2pliteral 3.91
set_literal 2.35
set_vtree_hashcode 1.82
free_vtree_cache 1.4
vtree_is_shannon_node 1.38
nnf_node_cmp 1.36
save_nnf_nodes 1.19
should_cache 1.13
match_keys 1.05
count_nnf_nodes 1.02
compile_dispatcher 0.95
0x00000000000a1747 0.76
qsort_r 0.76
literal2nnf_node 0.75
clist_size 0.74
_IO_file_xsputn 0.62
0x00000000001a0826 0.57
sat_mark_var 0.54
get_or_node 0.53
sat_var2nliteral 0.53
sat_unmark_var 0.53
copy_key 0.5
sat_is_instantiated_var 0.44
compile_vtree_shannon 0.41
0x000000000005a2ef 0.39
0x00000000000a28ca 0.39
insert_cache 0.38
var2nnf 0.36
erase_level 0.36
0x00000000000750c4 0.35
compile_vtree_decomposed 0.33
0x00000000000a285b 0.25
0x00000000000a174d 0.17
0x00000000000a18f2 0.17
new_and_node 0.16
conjoin_nnfs 0.16
vtree_shannon_var 0.15
0x0000000000075561 0.14
fprintf 0.14
sat_literal2var 0.13
sat_is_marked_var 0.13
unmark_vset 0.12
0x0000

In [80]:
# alternate method
# iterate over the callstack
# find the topmost function which is in target functions -> add to that fn

# target_functions_no_compile = set(filter(lambda x: not x.startswith("compile_") and not "mark" in x, target_functions))
target_functions_no_compile = target_functions

fn_pcts = {}
for fn in target_functions_no_compile:
    fn_pcts[fn] = 0

with open(PERF_FOLDED, "r") as f:
    lines = f.readlines()
    skip_section = False
    current_fn = None
    current_self_pct = 0
    for i, line in enumerate(lines):
        line = line.strip()
        if len(line) == 0:
            break
        if line[0] == "#":
            continue
        res = match_main_line(line)
        if res:
            if res["symbol"] not in target_functions_no_compile:
                skip_section = True
                continue
            if current_fn is not None:
                # callstack rows sometimes are less than the self time
                # 1. when there is no callstack
                # 2. maybe due to sampling?
                fn_pcts[current_fn] = max(fn_pcts[current_fn], current_self_pct)
            current_fn = res["symbol"]
            current_self_pct = res["self_pct"]
            skip_section = False
        else:
            if skip_section:
                continue
            pct, callstack = line.split(maxsplit=1)
            pct = float(pct.replace('%', ''))
            callstack = callstack.split(";")
            for fn in reversed(callstack):
                if fn in target_functions_no_compile:
                    if fn == current_fn:
                        fn_pcts[current_fn] += pct
                    break
fn_pcts = sorted(list(fn_pcts.items()), key=lambda x: x[0])
fn_pcts = { k: v for k, v in fn_pcts }
fn_pcts

{'allocate_manager_keys': 0,
 'allocate_vtree_keys': 0,
 'bits2bytes': 0,
 'c2d_version': 0,
 'clist_size': 0.74,
 'compile_dispatcher': 1.4600000000000009,
 'compile_vtree': 0,
 'compile_vtree_decomposed': 0.33,
 'compile_vtree_leaf': 0.03,
 'compile_vtree_shannon': 0.41,
 'compile_with_literal': 0,
 'construct_vtree_cache': 0,
 'construct_vtree_key': 21.61,
 'copy_key': 0.5,
 'count_dispatcher': 0,
 'count_vtree': 0,
 'count_vtree_decomposed': 0,
 'count_vtree_leaf': 0,
 'count_vtree_shannon': 0,
 'count_with_literal': 0,
 'drop_cache_entry': 0,
 'drop_vtree_cache_entries': 0,
 'extended_file_name': 0,
 'free_cache_entry': 0.01,
 'free_manager_keys': 0,
 'free_vtree_cache': 1.44,
 'free_vtree_keys': 0,
 'get_options': 0,
 'init_options': 0,
 'insert_cache': 0.38,
 'lookup_cache': 4.77,
 'main': 0,
 'match_keys': 1.05,
 'nnf_conjoin': 0.01,
 'nnf_count_models': 0,
 'nnf_count_nodes': 0,
 'nnf_decomposable': 0,
 'nnf_disjoin': 0,
 'nnf_edge_count': 0,
 'nnf_entails_cnf': 0,
 'nnf_free'

In [81]:
print(sum(fn_pcts.values()))

70.57


In [82]:
main_lines = []
sum_time = 0
for line in open(PERF_FOLDED, "r"):
    if line[0] == "#" or len(line.strip()) == 0:
        continue
    res = match_main_line(line)
    if res:
        print(sum_time)
        if len(main_lines) > 0:
            main_lines[-1]["sum_time"] = sum_time
        main_lines.append(res)
        print(res["symbol"], res["self_pct"], res["children_pct"], end=" ")
        sum_time = 0
    else:
        pct = line.split(maxsplit=1)[0]
        sum_time += float(pct.replace('%', ''))

0
compile_dispatcher 0.95 68.09 63.11000000000016
compile_vtree_shannon 0.41 68.09 63.11000000000016
compile_vtree_decomposed 0.33 68.09 63.11000000000016
lookup_cache 4.77 60.32 60.04
construct_vtree_key 21.61 21.62 21.390000000000008
sat_is_subsumed_clause 15.65 15.65 15.42999999999999
get_and_node 11.72 13.31 4.479999999999986
sat_is_implied_literal 10.77 10.77 10.449999999999998
0x000000002d7c4910 0.0 5.68 5.12999999999998
sat_var2pliteral 3.91 3.92 3.6699999999999995
set_literal 2.35 2.35 2.1199999999999988
0x00007fe30b0dfd90 0.0 2.2 2.1899999999999995
set_vtree_hashcode 1.82 1.82 1.7300000000000006
vtree_manager_free 0.0 1.46 1.45
free_vtree_cache 1.4 1.45 1.45
vtree_is_shannon_node 1.38 1.38 0.8600000000000004
nnf_node_cmp 1.36 1.36 1.36
save_nnf_nodes 1.19 1.19 1.19
should_cache 1.13 1.13 0.5800000000000003
match_keys 1.05 1.05 0.9200000000000004
count_nnf_nodes 1.02 1.02 0
insert_cache 0.38 0.81 0.5500000000000002
0x00007fe30b157747 0.0 0.76 0.76
0x00000000000a1747 0.76 0.76 0

In [83]:
for ml in main_lines:
    if ml["self_pct"] > ml.get("sum_time", 0):
        print(ml["symbol"], ml["self_pct"], ml["sum_time"], ml["children_pct"])


construct_vtree_key 21.61 21.390000000000008 21.62
sat_is_subsumed_clause 15.65 15.42999999999999 15.65
get_and_node 11.72 4.479999999999986 13.31
sat_is_implied_literal 10.77 10.449999999999998 10.77
sat_var2pliteral 3.91 3.6699999999999995 3.92
set_literal 2.35 2.1199999999999988 2.35
set_vtree_hashcode 1.82 1.7300000000000006 1.82
vtree_is_shannon_node 1.38 0.8600000000000004 1.38
should_cache 1.13 0.5800000000000003 1.13
match_keys 1.05 0.9200000000000004 1.05
count_nnf_nodes 1.02 0 1.02
qsort_r 0.76 0.04 0.76
literal2nnf_node 0.75 0.64 0.75
0x00000000001a0826 0.57 0.09 0.57
get_or_node 0.53 0.5 0.57
sat_mark_var 0.54 0.02 0.54
sat_var2nliteral 0.53 0.43000000000000016 0.53
sat_unmark_var 0.53 0.01 0.53
copy_key 0.5 0.4500000000000002 0.5
sat_is_instantiated_var 0.44 0.22000000000000003 0.44
0x00000000000a28ca 0.39 0.38 0.39
var2nnf 0.36 0.17000000000000004 0.36
erase_level 0.36 0.30000000000000004 0.36
0x00000000000a285b 0.25 0.24000000000000002 0.25
new_and_node 0.16 0.14 0.16
co

In [84]:
self_pcts = { ml["symbol"]: ml["self_pct"] for ml in main_lines }
self_pcts = sorted(list(self_pcts.items()), key=lambda x: x[0])
self_pcts = { k: v for k, v in self_pcts if not k.startswith('0x') }
self_pcts

{'Graph::containsEdge': 0.04,
 'Graph::removeAndConnect': 0.01,
 'Graph::removeEdge': 0.0,
 '_IO_do_write': 0.0,
 '_IO_file_overflow': 0.0,
 '_IO_file_write': 0.0,
 '_IO_file_xsputn': 0.62,
 '__default_morecore': 0.0,
 '__gmp_divide_by_zero': 0.0,
 '__gmp_doprnt': 0.0,
 '__gmp_doprnt_integer': 0.0,
 '__gmpf_get_str': 0.0,
 '__gmpn_dcpi1_bdiv_qr': 0.0,
 '__gmpn_dcpi1_div_qr': 0.0,
 '__gmpn_dcpi1_div_qr_n': 0.0,
 '__gmpn_dcpi1_divappr_q': 0.0,
 '__gmpn_get_str': 0.0,
 '__gmpn_hgcd2': 0.0,
 '__gmpn_hgcd_lehmer': 0.0,
 '__gmpn_hgcd_matrix_init': 0.0,
 '__gmpn_mul': 0.0,
 '__gmpn_mul_basecase': 0.0,
 '__gmpn_sbpi1_div_qr': 0.0,
 '__gmpn_sqr': 0.0,
 '__gmpn_sqr_basecase': 0.0,
 '__gmpn_sqrmod_bnm1': 0.0,
 '__gmpn_sub_n': 0.0,
 '__gmpn_tdiv_qr': 0.0,
 '__gmpn_toom44_mul': 0.0,
 '__gmpn_toom53_mul': 0.0,
 '__gmpn_toom6h_mul': 0.0,
 '__gmpn_toom8_sqr': 0.0,
 '__gmpn_toom8h_mul': 0.0,
 '__gmpn_toom_interpolate_12pts': 0.0,
 '__gmpq_get_str': 0.0,
 '__gmpz_sub': 0.0,
 '__libc_calloc': 0.04,
 '__o

In [85]:
nonzero_fn_pcts = { k: v for k, v in fn_pcts.items() if v > 0 }
nonzero_self_pcts = { k: v for k, v in self_pcts.items() if v > 0 }
perf_keys = set(nonzero_fn_pcts.keys())
all_keys = set(nonzero_self_pcts.keys())

difference_keys = all_keys - perf_keys
diff_pcts = { k: self_pcts[k] for k in difference_keys }

diff_pcts = sorted(list(diff_pcts.items()), key=lambda x: x[1], reverse=True)
diff_pcts = { k: v for k, v in diff_pcts }
total_diff_pcts = sum(diff_pcts.values())
print("total diff pcts", total_diff_pcts)
diff_pcts

total diff pcts 22.17


{'get_and_node': 11.72,
 'set_literal': 2.35,
 'nnf_node_cmp': 1.36,
 'save_nnf_nodes': 1.19,
 'count_nnf_nodes': 1.02,
 'qsort_r': 0.76,
 'literal2nnf_node': 0.75,
 '_IO_file_xsputn': 0.62,
 'get_or_node': 0.53,
 'erase_level': 0.36,
 'new_and_node': 0.16,
 'conjoin_nnfs': 0.16,
 'fprintf': 0.14,
 'vset_union': 0.12,
 'unmark_vset': 0.12,
 'mark_vset': 0.12,
 'cfree': 0.1,
 'remove_watched_clause': 0.08,
 'allocate_nnf_node': 0.07,
 'move_watched_literal': 0.07,
 'malloc': 0.05,
 '__libc_calloc': 0.04,
 'std::_Rb_tree_increment': 0.04,
 'Graph::containsEdge': 0.04,
 'cset_subset': 0.03,
 'fputc': 0.03,
 'sort_nnf_node': 0.03,
 'realloc': 0.02,
 'qsort': 0.02,
 'vset_intersection': 0.01,
 'eset_intersection': 0.01,
 'Graph::removeAndConnect': 0.01,
 'vset_free': 0.01,
 'fprintf@plt': 0.01,
 'disjoin_nnfs_i': 0.01,
 '__sbrk': 0.01}

In [86]:
# if i only took self time
target_self_pcts = { k: self_pcts[k] for k in perf_keys }
total_target_self_pcts = sum(target_self_pcts.values())

total_nonzero_fn_pcts = sum(nonzero_fn_pcts.values())

print("total target self pcts", total_target_self_pcts)
print("total fn pcts", total_nonzero_fn_pcts)

total target self pcts 70.02
total fn pcts 70.57


In [87]:
total_self_pcts = sum(self_pcts.values())
print("total self pcts", total_self_pcts)

total self pcts 92.19


In [88]:
minic2d_pcts = {}
for ml in main_lines:
    if ml["sharedobject"] != "miniC2D":
        continue
    minic2d_pcts[ml["symbol"]] = ml["self_pct"]
minic2d_pcts = sorted(list(minic2d_pcts.items()), key=lambda x: x[1], reverse=True)
minic2d_pcts = { k: v for k, v in minic2d_pcts }
minic2d_pcts

{'construct_vtree_key': 21.61,
 'sat_is_subsumed_clause': 15.65,
 'get_and_node': 11.72,
 'sat_is_implied_literal': 10.77,
 'lookup_cache': 4.77,
 'sat_var2pliteral': 3.91,
 'set_literal': 2.35,
 'set_vtree_hashcode': 1.82,
 'free_vtree_cache': 1.4,
 'vtree_is_shannon_node': 1.38,
 'nnf_node_cmp': 1.36,
 'save_nnf_nodes': 1.19,
 'should_cache': 1.13,
 'match_keys': 1.05,
 'count_nnf_nodes': 1.02,
 'compile_dispatcher': 0.95,
 'literal2nnf_node': 0.75,
 'clist_size': 0.74,
 'sat_mark_var': 0.54,
 'get_or_node': 0.53,
 'sat_var2nliteral': 0.53,
 'sat_unmark_var': 0.53,
 'copy_key': 0.5,
 'sat_is_instantiated_var': 0.44,
 'compile_vtree_shannon': 0.41,
 'insert_cache': 0.38,
 'var2nnf': 0.36,
 'erase_level': 0.36,
 'compile_vtree_decomposed': 0.33,
 'new_and_node': 0.16,
 'conjoin_nnfs': 0.16,
 'vtree_shannon_var': 0.15,
 'sat_literal2var': 0.13,
 'sat_is_marked_var': 0.13,
 'unmark_vset': 0.12,
 'vset_union': 0.12,
 'mark_vset': 0.12,
 'sat_is_irrelevant_var': 0.1,
 'vtree_is_leaf': 0.1,

In [89]:
total_minic2d_pcts = sum(minic2d_pcts.values())
print("total minic2d pcts", total_minic2d_pcts)

total minic2d pcts 90.36


In [90]:
non_minic2d_pcts = {}
for ml in main_lines:
    if ml["sharedobject"] == "miniC2D":
        continue
    non_minic2d_pcts[ml["symbol"]] = ml["self_pct"]
non_minic2d_pcts = sorted(list(non_minic2d_pcts.items()), key=lambda x: x[1], reverse=True)
non_minic2d_pcts = { k: v for k, v in non_minic2d_pcts }
non_minic2d_pcts

{'0x00000000000a1747': 0.76,
 'qsort_r': 0.76,
 '_IO_file_xsputn': 0.62,
 '0x00000000001a0826': 0.57,
 '0x000000000005a2ef': 0.39,
 '0x00000000000a28ca': 0.39,
 '0x00000000000750c4': 0.35,
 '0x00000000000a285b': 0.25,
 '0x00000000000a174d': 0.17,
 '0x00000000000a18f2': 0.17,
 '0x0000000000075561': 0.14,
 'fprintf': 0.14,
 '0x000000000007539c': 0.12,
 'cfree': 0.1,
 '0x0000000000075247': 0.1,
 '0x000000000019d43c': 0.09,
 '0x000000000005a2dd': 0.09,
 '0x00000000000444bd': 0.08,
 '0x00000000001a0830': 0.08,
 '0x00000000001a081b': 0.07,
 '0x00000000000283e4': 0.07,
 '0x00000000001a0845': 0.07,
 '0x00000000000750cc': 0.06,
 '0x00000000001a07f3': 0.06,
 'malloc': 0.05,
 '0x000000000004437d': 0.05,
 '0x00000000000444c4': 0.05,
 '0x0000000000075178': 0.05,
 '0x0000000000044404': 0.05,
 '0x0000000000044467': 0.04,
 'std::_Rb_tree_increment': 0.04,
 '0x00000000000443a9': 0.04,
 '0x0000000000075096': 0.04,
 '0x000000000004436b': 0.04,
 '0x000000000019d436': 0.04,
 '0x0000000000044455': 0.04,
 '0

In [91]:
total_nonminic2d_pcts = sum(non_minic2d_pcts.values())
print("total nonminic2d pcts", total_nonminic2d_pcts)

total nonminic2d pcts 8.68
